# Comparing RT-Ehrenfest and ASE/Psi4 Trajectories

"
            "This walkthrough cross-validates MaxwellLink's real-time Ehrenfest integration"
            " against the Born–Oppenheimer dynamics available through ASE with the Psi4"
            " calculator. We propagate a short HCN trajectory with both pathways and"
            " confirm that the nuclear motion matches within numerical noise.

**Prerequisites**

"
            "- `psi4` and the Psi4 Python bindings
"
            "- `ase` with the Psi4 calculator plugin
"
            "- `matplotlib` for plotting
"
            "- access to `tests/data/hcn.xyz`, which ships with the MaxwellLink repository

"
            "The steps below mirror the configuration used in the automated regression tests.

## 1. Import dependencies and locate the geometry

"
            "We begin by importing the required libraries and verifying that the reference"
            " XYZ file is available. Positions returned by ASE are expressed in ångström,"
            " so we also define the ångström→bohr conversion factor used throughout the"
            " comparison.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import maxwelllink as mxl
from pathlib import Path

ANGSTROM_TO_BOHR = 1.8897259886
XYZ_PATH = Path('tests/data/hcn.xyz').resolve()
if not XYZ_PATH.exists():
    raise FileNotFoundError(f'Could not find the HCN geometry at {XYZ_PATH}')

print(f'Geometry file: {XYZ_PATH}')

## 2. Configure the RT-Ehrenfest driver

"
            "The `RTEhrenfestModel` embeds Psi4 to propagate electrons and nuclei on the same"
            " time grid. We request a modest B3LYP/STO-3G setup with Born–Oppenheimer forces"
            " (`force_type='bo'`) so the results are directly comparable to the ASE trajectory."
            " Initialization performs the SCF warm-up and prepares the nuclear integrator.

In [ ]:
model_rt = mxl.RTEhrenfestModel(
    engine='psi4',
    molecule_xyz=str(XYZ_PATH),
    functional='b3lyp',
    basis='sto-3g',
    dt_rttddft_au=10.0,
    delta_kick_au=0.0,
    memory='2GB',
    verbose=False,
    remove_permanent_dipole=False,
    n_fock_per_nuc=1,
    n_elec_per_fock=1,
    homo_to_lumo=False,
    force_type='bo',
    partial_charges=[1.0, -1.0, 0.0],
)
model_rt.initialize(dt_new=10.0, molecule_id=0)

print(f'RT-Ehrenfest nuclear time step dt_N = {model_rt.dtN:.3f} a.u.')

## 3. Configure the ASE/Psi4 Born–Oppenheimer driver

"
            "The ASE pathway evaluates the same B3LYP/STO-3G forces at each time step."
            " MaxwellLink wraps the calculator through `ASEModel`, exposing a driver interface"
            " compatible with the Ehrenfest comparison.

In [ ]:
model_ase = mxl.ASEModel(
    atoms=str(XYZ_PATH),
    calculator='psi4',
    calc_kwargs='method=b3lyp, basis=sto-3g',
    charges='[1.0 -1.0 0.0]',
)
model_ase.initialize(dt_new=10.0, molecule_id=0)

initial_snapshot = model_ase._snapshot()
print('Initial ASE positions (Å):')
print(initial_snapshot['positions'])

## 4. Propagate both models under the same external field

"
            "We now apply a weak static electric field to both dynamics engines. Each call"
            " to `propagate` advances the nuclei by one Ehrenfest time step. We record the"
            " C–H bond length after every step in both unit systems.

In [ ]:
n_steps = 10
efield_vec = np.array([1e-2, 1e-2, 0.0])  # in atomic units

time_axis_au = [0.0]
traj_bond_rt = [np.linalg.norm(model_rt.traj_R[0][0] - model_rt.traj_R[0][1])]
traj_bond_ase = [
    np.linalg.norm(initial_snapshot['positions'][0] - initial_snapshot['positions'][1]) * ANGSTROM_TO_BOHR
]

for step in range(1, n_steps + 1):
    model_rt.propagate(effective_efield_vec=efield_vec)
    positions_rt = model_rt.traj_R[-1]

    model_ase.propagate(effective_efield_vec=efield_vec)
    snapshot = model_ase._snapshot()
    positions_ase_bohr = snapshot['positions'] * ANGSTROM_TO_BOHR

    time_axis_au.append(step * model_rt.dtN)
    traj_bond_rt.append(np.linalg.norm(positions_rt[0] - positions_rt[1]))
    traj_bond_ase.append(np.linalg.norm(positions_ase_bohr[0] - positions_ase_bohr[1]))

## 5. Compare the resulting bond dynamics

"
            "Converting the time axis to femtoseconds and plotting the C–H bond length from"
            " each integrator reveals the expected agreement. We also quote the maximum"
            " absolute deviation between the two trajectories.

In [ ]:
time_fs = np.array(time_axis_au) * 0.02418884254
bond_rt = np.array(traj_bond_rt)
bond_ase = np.array(traj_bond_ase)

max_dev = np.max(np.abs(bond_rt - bond_ase))
print(f'Maximum |Δ bond| = {max_dev:.3e} bohr')

plt.figure(figsize=(6, 4))
plt.plot(time_fs, bond_rt, label='RT-Ehrenfest (Psi4)')
plt.plot(time_fs, bond_ase, '--', label='ASE (Psi4)')
plt.xlabel('time (fs)')
plt.ylabel('C–H bond length (bohr)')
plt.legend()
plt.tight_layout()
plt.show()

### Wrap-up

"
            "Both propagation strategies track each other to within machine precision,"
            " confirming that MaxwellLink's Ehrenfest implementation is consistent with the"
            " ASE + Psi4 reference for this benchmark system.